In [178]:
from deck import full_euchre_deck
from play_round import PlayRound
from dealer import Dealer
import pandas as pd
import numpy as np

In [179]:
game = Dealer(deck=full_euchre_deck ,players=4)
game.deal_cards()
hands5 = [game.hand1, game.hand2, game.hand3, game.hand4]
hands5
# first_play = [game.hand1[0], game.hand1[1], game.hand1[2], game.hand1[3], game.hand1[4]]

[array([[ 10,   0],
        [  0,  90],
        [  0, -13],
        [ 13,   0],
        [ 11,   0]]),
 array([[   0,  120],
        [   0, -135],
        [ -13,    0],
        [  12,    0],
        [   0,  -12]]),
 array([[-10,   0],
        [-11,   0],
        [  0, 140],
        [  0, 130],
        [  0, -11]]),
 array([[-14,   0],
        [-12,   0],
        [  0, 100],
        [  0, -10],
        [  9,   0]])]

In [272]:
score = []
round2_hands = []

In [273]:
for i in range(5):
    round1 = PlayRound(hands=hands5, lead=0, card_play=i)
    for j in range(125):
        try: 
            round1.play_round(bit=j)
            round2_hands.append(round1.next_round_hands)
            score.append(round1.round_winner)

        except IndexError:
            break

In [274]:
len(round2_hands), len(score)

(20, 20)

In [275]:
round2_score = []
round3_hands = []
r2_full_res = []

In [276]:
for i, j in zip(round2_hands, score):
    for k in range(4):
        round2 = PlayRound(hands=i, lead=j, card_play=k)
        for m in range(64):
            try: 
                round2.play_round(bit=m)
                round3_hands.append(round2.next_round_hands)
                round2_score.append(round2.round_winner)
                r2_full_res.append([j, round2.round_winner])

            except IndexError:
                break

In [277]:
len(round2_score), len(round3_hands)

(776, 776)

In [278]:
round3_score = []
round4_hands = []
r3_full_res = []

In [279]:
for h, i, j in zip(r2_full_res, round3_hands, round2_score):
    for k in range(3):
        round3 = PlayRound(hands=i, lead=j, card_play=k)
        for m in range(27):
            try: 
                round3.play_round(bit=m)
                round4_hands.append(round3.next_round_hands)
                round3_score.append(round3.round_winner)
                r3_full_res.append([h[0], h[1], round3.round_winner])

            except IndexError:
                break

In [280]:
len(round3_score)

15471

In [281]:
round4_score = []
round5_hands = []
r4_full_res = []

In [282]:
for h, i, j in zip(r3_full_res, round4_hands, round3_score):
    for k in range(2):
        round4 = PlayRound(hands=i, lead=j, card_play=k)
        for m in range(8):
            try: 
                round4.play_round(bit=m)
                round5_hands.append(round4.next_round_hands)
                round4_score.append(round4.round_winner)
                r4_full_res.append([h[0], h[1], h[2], round4.round_winner])

            except IndexError:
                break

In [283]:
len(round4_score)

107568

In [284]:
round5_score = []
r5_full_res = []

In [285]:
for h, i, j in zip(r4_full_res, round5_hands, round4_score):
    round5 = PlayRound(hands=i, lead=j, card_play=0)
    round5.play_round(bit=0)
    round5_score.append(round5.round_winner)
    r5_full_res.append([h[0], h[1], h[2], h[3], round5.round_winner])

In [286]:
len(round5_score)

107568

In [287]:
len(score)+len(round2_score)+len(round3_score)+len(round4_score)+len(round5_score)

231403

In [288]:
res_df = pd.DataFrame(r5_full_res, columns=['r1_winner', 'r2_winner', 'r3_winner', 'r4_winner', 'r5_winner'])

In [289]:
res_df.columns

Index(['r1_winner', 'r2_winner', 'r3_winner', 'r4_winner', 'r5_winner'], dtype='object')

In [290]:
cols = ['r1_winner', 'r2_winner', 'r3_winner', 'r4_winner', 'r5_winner']

for i in cols:
    res_df[f'{i}_team'] = res_df[i] % 2

In [291]:
res_df.columns

Index(['r1_winner', 'r2_winner', 'r3_winner', 'r4_winner', 'r5_winner',
       'r1_winner_team', 'r2_winner_team', 'r3_winner_team', 'r4_winner_team',
       'r5_winner_team'],
      dtype='object')

In [292]:
res_df['points'] = res_df['r1_winner_team'] + res_df['r2_winner_team'] + res_df['r3_winner_team'] + res_df['r4_winner_team'] + res_df['r5_winner_team']

In [293]:
res_df['winner'] = np.where(res_df['points']>2, 1, 0)

In [294]:
res_df['winner'].mean()

0.26271753681392235